In [1]:
from data_processor_lib import Processor

data_src = "https://media.geeksforgeeks.org/wp-content/uploads/nba.csv"

In [2]:
processor = Processor()
processor.read_csv(data_src)

Profiler initialized


In [3]:
processor.columns()

Index(['Name', 'Team', 'Number', 'Position', 'Age', 'Height', 'Weight',
       'College', 'Salary'],
      dtype='object')

In [4]:
processor.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0


In [5]:
processor.overview()

,Value
Total Columns,9.0
Total Rows,458.0
Incomplete Rows Count,NaN
Incomplete Rows (%),NaN
Incomplete Columns Count,NaN
Total Data Points,NaN
Incomplete Data Points Count,NaN
Incomplete Data (%),NaN


In [6]:
processor.insights()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
Valid Data Count,457.0,457.0,457.0,457.0,457.0,457.0,457.0,373.0,446.0
Valid Data (%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Missing Data Count,1.0,1.0,1.0,1.0,1.0,1.0,1.0,85.0,12.0
Missing Data (%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Data Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Median,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Standard Deviation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
